<a href="https://colab.research.google.com/github/gustavocwl/vacinacovidparana/blob/main/vacina_covid_pr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas

In [3]:
import pandas as pd
import numpy as np

from google.colab import drive
from google.colab import files
drive.mount('/content/drive/', force_remount = False)

Mounted at /content/drive/


In [4]:
import gspread
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
from gspread_dataframe import set_with_dataframe

In [5]:
# WEB SCRAPING LINK ARQUIVO CSV NO SITE OPENDATASUS
import requests
from bs4 import BeautifulSoup
url = 'https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8'
html = requests.get(url)
bs_obj = BeautifulSoup (html.text,'lxml')
h1 = bs_obj.find({'h1':'URL: '})
lista = h1.find_next().find_all("a")
url_csv = lista[0].text

In [ ]:
# LEITURA VACINA
df = pd.read_csv(url_csv,
                 usecols=['vacina_dataAplicacao','paciente_idade','paciente_enumSexoBiologico','paciente_racaCor_valor','vacina_grupoAtendimento_nome','vacina_categoria_nome','vacina_nome','vacina_descricao_dose','estabelecimento_municipio_codigo','estabelecimento_uf'],
                 dtype={'estabelecimento_municipio_codigo':str},
                 sep='"',
                 delimiter=',',
                 encoding='UTF-8',
                 low_memory=False,
                 skip_blank_lines=True,
                 error_bad_lines=False,
                 parse_dates=['vacina_dataAplicacao'],
                 chunksize=20000)
df = pd.concat(df)

In [ ]:
# SELECIONAR DOSES APLICADAS NO PARANÁ
vacinapr = df.loc[df['estabelecimento_uf'] == 'PR'].copy()

# FORMATAR DATA
vacinapr['vacina_dataAplicacao'] = vacinapr['vacina_dataAplicacao'].astype(str).str[0:10]
vacinapr['vacina_dataAplicacao'] = pd.to_datetime(vacinapr['vacina_dataAplicacao'], format='%Y/%m/%d')

# CATEGORIZAR IDADE EM FAIXA ETÁRIA
vacinapr.loc[vacinapr['paciente_idade'] >=80, 'Faixa Etária'] = '80 anos e mais'
vacinapr.loc[vacinapr['paciente_idade'] <80, 'Faixa Etária'] = '70 a 79 anos'
vacinapr.loc[vacinapr['paciente_idade'] <70, 'Faixa Etária'] = '60 a 69 anos'
vacinapr.loc[vacinapr['paciente_idade'] <60, 'Faixa Etária'] = '50 a 59 anos'
vacinapr.loc[vacinapr['paciente_idade'] <50, 'Faixa Etária'] = '40 a 49 anos'
vacinapr.loc[vacinapr['paciente_idade'] <40, 'Faixa Etária'] = '30 a 39 anos'
vacinapr.loc[vacinapr['paciente_idade'] <30, 'Faixa Etária'] = '20 a 29 anos'
vacinapr.loc[vacinapr['paciente_idade'] <20, 'Faixa Etária'] = '10 a 19 anos'
vacinapr.loc[vacinapr['paciente_idade'] <10, 'Faixa Etária'] = '1 a 0 anos'

# CONTAR REGISTROS INDIVIDUAIS
vacinapr['contar'] = 1

# PIVOT TABLE FAIXA ETARIA
vacinados_mun = pd.pivot_table(vacinapr , index=['estabelecimento_municipio_codigo'] , columns=['Faixa Etária'] , values=['contar'] , aggfunc='sum' , fill_value=0 , margins=True , margins_name='Total')

vacinados_mun_excel = vacinados_mun.to_excel('vacinados_mun_excel.xls')
vacinados_mun_excel = pd.read_excel('vacinados_mun_excel.xls',skiprows=2,skipfooter=1,dtype={'estabelecimento_municipio_codigo':str})

vacinados_mun = vacinados_mun_excel.rename(columns={'estabelecimento_municipio_codigo': 'CodMun6',
                                                          'Unnamed: 1': 'vac_0-9',
                                                          'Unnamed: 2': 'vac_10-19',
                                                          'Unnamed: 3': 'vac_20-29',
                                                          'Unnamed: 4': 'vac_30-39',
                                                          'Unnamed: 5': 'vac_40-49',
                                                          'Unnamed: 6': 'vac_50-59',
                                                          'Unnamed: 7': 'vac_60-69',
                                                          'Unnamed: 8': 'vac_70-79',
                                                          'Unnamed: 9': 'vac_80',
                                                          'Unnamed: 10': 'vac_total',
                                                          })

# PIVOT TABLE CATEGORIA
vacinapr['vacina_categoria_nome'] = df['vacina_categoria_nome'].fillna(value='Em branco')

mun_categoria = pd.pivot_table(vacinapr , index=['estabelecimento_municipio_codigo'] , columns=['vacina_categoria_nome'] , values=['contar'] , aggfunc='count' , fill_value=0 , margins=True , margins_name='Total')

mun_categoria_excel = mun_categoria.to_excel('mun_categoria_excel.xls')
mun_categoria_excel = pd.read_excel('mun_categoria_excel.xls',skiprows=[2],skipfooter=1,dtype={'vacina_categoria_nome':str} , header=1)

mun_categoria = mun_categoria_excel.rename(columns={'vacina_categoria_nome': 'CodMun6'})

# PIVOT TABLE DOSE
mun_dose = pd.pivot_table(vacinapr , index=['estabelecimento_municipio_codigo'] , columns=['vacina_descricao_dose'] , values=['contar'] , aggfunc='count' , fill_value=0 , margins=True , margins_name='Total')

mun_dose_excel = mun_dose.to_excel('mun_dose_excel.xls')
mun_dose_excel = pd.read_excel('mun_dose_excel.xls',skiprows=[2],skipfooter=1,dtype={'vacina_descricao_dose':str} , header=1)

mun_dose = mun_dose_excel.rename(columns={'vacina_descricao_dose': 'CodMun6',
                                          '\xa0\xa0\xa0\xa01ª\xa0Dose': 'primeira_dose',
                                          '\xa0\xa0\xa0\xa02ª\xa0Dose': 'segunda_dose',
                                          '\xa0\xa0\xa0\xa0Única\xa0': 'unica_dose',
                                          'Total': 'total_dose'})

In [ ]:
# LEITURA INFO MUNICIPIOS
url_mun_info = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSklorqgvstUBOt1WBEtkBaRzPGzddNx4A-FUMOSOVxps0fQ4XtT8rPyYNVA7DRQe-6c5nY65cuND88/pub?output=xls'
mun_info = pd.read_excel(url_mun_info,
                         dtype={'CodMun6':str})

In [ ]:
# REGISTROS INDIVIDUAIS DE VACINAÇÃO PARANÁ

# SELECIONAR DOSES APLICADAS NO PARANÁ
vacina_covid_pr_individual = df.loc[df['estabelecimento_uf'] == 'PR'].copy()

# SELECIONAR COLUNAS
vacina_covid_pr_individual = vacina_covid_pr_individual[['vacina_dataAplicacao',
                                                         'paciente_idade',
                                                         'paciente_enumSexoBiologico',
                                                         'paciente_racaCor_valor',
                                                         'vacina_grupoAtendimento_nome',
                                                         'vacina_categoria_nome',
                                                         'vacina_nome',
                                                         'vacina_descricao_dose',
                                                         'estabelecimento_municipio_codigo']].copy()

# FORMATAR CAMPOS
vacina_covid_pr_individual['CodMun6'] = vacina_covid_pr_individual['estabelecimento_municipio_codigo'].apply(str)
vacina_covid_pr_individual['vacina_dataAplicacao'] = vacina_covid_pr_individual['vacina_dataAplicacao'].astype(str).str[0:10]
vacina_covid_pr_individual['vacina_dataAplicacao'] = pd.to_datetime(vacina_covid_pr_individual['vacina_dataAplicacao'], format='%Y/%m/%d')

# CATEGORIZAR IDADE EM FAIXA ETÁRIA
vacina_covid_pr_individual.loc[vacina_covid_pr_individual['paciente_idade'] >=80, 'Faixa Etária'] = '80 anos e mais'
vacina_covid_pr_individual.loc[vacina_covid_pr_individual['paciente_idade'] <80, 'Faixa Etária'] = '70 a 79 anos'
vacina_covid_pr_individual.loc[vacina_covid_pr_individual['paciente_idade'] <70, 'Faixa Etária'] = '60 a 69 anos'
vacina_covid_pr_individual.loc[vacina_covid_pr_individual['paciente_idade'] <60, 'Faixa Etária'] = '50 a 59 anos'
vacina_covid_pr_individual.loc[vacina_covid_pr_individual['paciente_idade'] <50, 'Faixa Etária'] = '40 a 49 anos'
vacina_covid_pr_individual.loc[vacina_covid_pr_individual['paciente_idade'] <40, 'Faixa Etária'] = '30 a 39 anos'
vacina_covid_pr_individual.loc[vacina_covid_pr_individual['paciente_idade'] <30, 'Faixa Etária'] = '20 a 29 anos'
vacina_covid_pr_individual.loc[vacina_covid_pr_individual['paciente_idade'] <20, 'Faixa Etária'] = '10 a 19 anos'
vacina_covid_pr_individual.loc[vacina_covid_pr_individual['paciente_idade'] <10, 'Faixa Etária'] = '1 a 0 anos'

# ORDENAR COLUNAS
vacina_covid_pr_individual = vacina_covid_pr_individual[['vacina_dataAplicacao',
                                                         'paciente_idade',
                                                         'Faixa Etária',
                                                         'paciente_enumSexoBiologico',
                                                         'paciente_racaCor_valor',
                                                         'vacina_grupoAtendimento_nome',
                                                         'vacina_categoria_nome',
                                                         'vacina_nome',
                                                         'vacina_descricao_dose',
                                                         'CodMun6']].copy()

# VINCULAR INFORMAÇÕES DO MUNICÍPIO
mun_info_individual = mun_info[['CodMun6',
                                'NomeMun',
                                'NomRegSau',
                                'porte']].copy()

# CodMun6 string
mun_info_individual['CodMun6'] = mun_info_individual['CodMun6'].apply(str)

vacina_covid_pr_individual2 = pd.merge(vacina_covid_pr_individual, mun_info_individual, on='CodMun6')

vacina_covid_pr_individual['vacina_dataAplicacao'] = pd.to_datetime(vacina_covid_pr_individual['vacina_dataAplicacao'], format='%d/%m/%Y')

# EXPORTAR GOOGLE SHEETS
sheet_vacina_covid_pr_individual = gc.open_by_key('1VcdGwVDSwlT4VqFHjcHVmOsfcWe_Z9lP3swcKdR0-BE').sheet1
sheet_vacina_covid_pr_individual.clear()
set_with_dataframe(sheet_vacina_covid_pr_individual, vacina_covid_pr_individual2,resize=True)

In [ ]:
# VINCULAR INFORMAÇÕES DO MUNICÍPIO
dfagrupado = pd.merge(mun_info, vacinados_mun, on=['CodMun6'], how='right')
dfagrupado = pd.merge(dfagrupado, mun_categoria, on=['CodMun6'], how='outer')
dfagrupado = pd.merge(dfagrupado, mun_dose, on=['CodMun6'], how='outer')

# SOMAR POPULAÇÃO FAIXA ETÁRIAS
dfagrupado['pop_70-79'] = dfagrupado['pop_70-74'] + dfagrupado['pop_75-79']
dfagrupado['pop_60-69'] = dfagrupado['pop_60-64'] + dfagrupado['pop_65-69']
dfagrupado['pop_50-59'] = dfagrupado['pop_50-54'] + dfagrupado['pop_55-59']
dfagrupado['pop_40-49'] = dfagrupado['pop_40-44'] + dfagrupado['pop_45-49']
dfagrupado['pop_30-39'] = dfagrupado['pop_30-34'] + dfagrupado['pop_35-39']
dfagrupado['pop_20-29'] = dfagrupado['pop_20-24'] + dfagrupado['pop_25-29']
dfagrupado['pop_10-19'] = dfagrupado['pop_10-14'] + dfagrupado['pop_15-19']
dfagrupado['pop_0-9'] = dfagrupado['pop_0-4'] + dfagrupado['pop_5-9']

# CALCULAR COBERTURA VACINAL
dfagrupado['cv_0-9'] = ( dfagrupado['vac_0-9'] / dfagrupado['pop_0-9'] ) * 100
dfagrupado['cv_10-19'] = ( dfagrupado['vac_10-19'] / dfagrupado['pop_10-19'] ) * 100
dfagrupado['cv_20-29'] = ( dfagrupado['vac_20-29'] / dfagrupado['pop_20-29'] ) * 100
dfagrupado['cv_30-39'] = ( dfagrupado['vac_30-39'] / dfagrupado['pop_30-39'] ) * 100
dfagrupado['cv_40-49'] = ( dfagrupado['vac_40-49'] / dfagrupado['pop_40-49'] ) * 100
dfagrupado['cv_50-59'] = ( dfagrupado['vac_50-59'] / dfagrupado['pop_50-59'] ) * 100
dfagrupado['cv_60-69'] = ( dfagrupado['vac_60-69'] / dfagrupado['pop_60-69'] ) * 100
dfagrupado['cv_70-79'] = ( dfagrupado['vac_70-79'] / dfagrupado['pop_70-79']) * 100
dfagrupado['cv_80'] = ( dfagrupado['vac_80'] / dfagrupado['pop_80'] ) * 100

# EXPORTAR GOOGLE SHEETS
sheet_vacina_covid_pr = gc.open_by_key('1W2VYW63iJnHrLWJSZADIu-H6tgpOmPU5YQEvFH_Dm6w').sheet1
sheet_vacina_covid_pr.clear()
set_with_dataframe(sheet_vacina_covid_pr, dfagrupado,resize=True)

In [ ]:
# RELATÓRIO FINAL
print('Número de vacinados no Brasil:', len(df.index))
print('Número de vacinados no Paraná:', len(vacina_covid_pr_individual.index))
print('Número de municípios do Paraná que enviaram dados:', len(dfagrupado.index))
print('Data da última dose aplicada no PR:', vacina_covid_pr_individual['vacina_dataAplicacao'].max().strftime("%d/%m/%Y"))

In [ ]:
# WEB SCRAPING DATA DA ÚLTIMA ATUALIZAÇÃO
url2 = 'https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao'
html2 = requests.get(url2)
bs_obj2 = BeautifulSoup (html2.text,'lxml')
tr = bs_obj2.find({'tbody':'Última Atualização'})
lista2 = tr.find_next().find_all("td")
ultima_atualizacao = lista2[0].text
ultima_atualizacao = ultima_atualizacao.replace('\n','')
ultima_atualizacao = ultima_atualizacao.replace(' (UTC)','')
ultima_atualizacao = ['Data',ultima_atualizacao]
ultima_atualizacao = pd.DataFrame(ultima_atualizacao)
ultima_atualizacao = ultima_atualizacao.rename(columns={0: 'Data'})
ultima_atualizacao = ultima_atualizacao.iloc[1:]

# EXPORTAR GOOGLE SHEETS
sheet_ultima_atualizacao = gc.open_by_key('1GURxzVra7L0o-O4zBI4MpoT66dEESFHq6BYiZ_n3Buc').get_worksheet(1)
set_with_dataframe(sheet_ultima_atualizacao, ultima_atualizacao,resize=False)